<a href="https://colab.research.google.com/github/texaschikkita/deep-learning-challenge/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [50]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [51]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis = 1)

In [53]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [54]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 0.05 * len(application_df)
cutoff_value




1714.95

In [56]:
# use variable name 'application_types_to_replace' 
application_types_to_replace = list(application_df["APPLICATION_TYPE"].value_counts() < cutoff_value)

In [57]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [58]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [59]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [60]:
# look at classification counts more than 10 as well to compare data
classification_counts[classification_counts > 10]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [61]:
# use cutoff_value from above
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [62]:
# copy the dataframe to preserve and work. 
encoded_df = application_df.copy()

In [64]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(application_df)

In [65]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop(["IS_SUCCESSFUL"], axis = "columns").values
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)

In [66]:
# try with a different random_state to compare (comment out the one not using so not to confuse yourself or error out)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=5)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [41]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer
# # Ideally there should be as many outputs as there are neurons.  If less is set the network will need more hidden networks to abstract the data even more to represent it in a more compact form, 
# and data needs to be represented in binary which means by default the neural network needs an additional hidden layer to decompose data even more.


# number_input_features = len(X_train[0])
# hidden_nodes_layer1 =  8
# hidden_nodes_layer2 = 5

# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=2, activation="softmax"))

# # Check the structure of the model
# nn.summary()

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer
# sigmoid used for binary classification in logistic_regression_model.  While creating artificial neurons it's the activation function.  In statistics the sigmoid functions in the cumulative distribution function = sigmoid' 
#softmax = multi-classification in Logistic Regression Model.  In building neural networks it's activation function used in different layer level. They are similar.
# tested both to see variation, if any.  unable to detect differences in function yet in the activity. 
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 80)                4160      
                                                                 
 dense_22 (Dense)            (None, 30)                2430      
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,621
Trainable params: 6,621
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics =["accuracy"])

In [70]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5688 - accuracy: 0.7222
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7279
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7326
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5440 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5443 - accura

In [71]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5649 - accuracy: 0.7287 - 397ms/epoch - 1ms/step
Loss: 0.5648588538169861, Accuracy: 0.7287463545799255


In [72]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")